### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data = "../../UR-RICH-DATA-PT-01-2020-U-C/04-Pandas/HW/Instructions/PyCitySchools/Resources/schools_complete.csv"
student_data= "../../UR-RICH-DATA-PT-01-2020-U-C/04-Pandas/HW/Instructions/PyCitySchools/Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data_df = pd.read_csv(school_data)
student_data_df = pd.read_csv(student_data)

# Combine the data into a single dataset
complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name"])
complete_df


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
school_students = complete_df["school_name"].value_counts()
total_schools = len(school_students)

total_students = school_students.sum()

school_budgets = complete_df["budget"].unique()
total_budget = school_budgets.sum()

district_math_scores = complete_df["math_score"]
district_average_math = round(district_math_scores.mean(),2)

district_read_scores = complete_df["reading_score"]
district_average_read = round(district_read_scores.mean(),2)

district_passing_rate = ((district_average_math + district_average_read)/2)


district_passing_math = complete_df.loc[complete_df["math_score"] >= 70 ]
district_passing_math_amount = len(district_passing_math)
district_percent_math = (district_passing_math_amount/len(complete_df)) * 100


district_passing_read = complete_df.loc[complete_df["reading_score"] >= 70 ]
district_passing_read_amount = len(district_passing_read)
district_percent_read = (district_passing_read_amount/len(complete_df)) * 100


district_summary_dict = {"Total Schools": [total_schools], "Total Students": [total_students], "Total Budget": [total_budget], 
                         "Average Math Score": [district_average_math],"Average Reading Score": [district_average_read], 
                         "% Passing Math": [district_percent_math],"% Passing Reading": [district_percent_read], 
                         "% Overall Passing Rate": [district_passing_rate]}
district_summary_df = pd.DataFrame(district_summary_dict)
district_summary_df = district_summary_df.style.format({"Total Students": "{:,}","Total Budget": "${:,.2f}",
                                                        "Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}", 
                                                        "% Passing Math": "{0:.2f}%", "% Passing Reading": "{0:.2f}%", 
                                                        "% Overall Passing Rate": "{0:.2f}%"})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.44%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:

'''School Type'''
#make table of repeats
school_types = complete_df[["school_name", "type"]]
#drop duplicate schools to get list of each schools type
school_types = school_types.drop_duplicates("school_name")
#index by school name to isolate types into a series
school_types = school_types.set_index("school_name")
#store type column
school_types_list = school_types["type"]

'''School Name'''
#groupby school name (creates groupby object - must perform functions to see)
school_groupby = complete_df.groupby(["school_name"])

'''Total School Budget, 
Average Math Score, 
Average Reading Score'''

#get means of all numeric values
school_means = school_groupby.mean()
#grab relevant columns
school_average_read = school_means["reading_score"]
school_average_math = school_means["math_score"]
school_budget = school_means["budget"]

'''Total Students'''
#calculated in district summary

'''Per Student Budget'''
school_budget_per_student = school_budget/school_students

'''% Passing Math
%Passing Reading
% Overall Passing'''

#total scores
school_reading_totals = school_groupby["reading_score"].count()
school_math_totals = school_groupby["math_score"].count()

#passing scores
grouped_school_passing_read = district_passing_read.groupby("school_name")
grouped_school_passing_math = district_passing_math.groupby("school_name")
school_passing_read = grouped_school_passing_read["reading_score"].count()
school_passing_math = grouped_school_passing_math["reading_score"].count()

#percentages
school_percent_read = (school_passing_read/school_reading_totals) * 100
school_percent_math = (school_passing_math/school_math_totals) * 100
school_overall_passing = (school_percent_read + school_percent_math)/2

'''Summary Table'''
school_summary_df = pd.DataFrame ({"School Type": school_types_list,
                                   "Total Students": school_students,
                                   "Total School Budget": school_budget,
                                   "Per Student Budget": school_budget_per_student,
                                  "Average Math Score": school_average_math,
                                 "Average Reading Score": school_average_read,
                                  "% Passing Math": school_percent_math,
                                  "% Passing Reading": school_percent_read,
                                  "% Overall Passing Rate": school_overall_passing})


school_summary_formatted = school_summary_df.style.format({"Total Students": "{:,}","Total School Budget": "${:,.2f}", 
                                                    "Per Student Budget": "${:,.2f}",
                                                        "Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}", 
                                                        "% Passing Math": "{0:.2f}%", "% Passing Reading": "{0:.2f}%", 
                                                        "% Overall Passing Rate": "{0:.2f}%"})

In [4]:
top_schools = school_summary_df.sort_values(by=["% Overall Passing Rate"], ascending=False)
top_schools = top_schools.iloc[0:5,:]
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [5]:
bottom_schools = school_summary_df.sort_values(by=["% Overall Passing Rate"])
bottom_schools =bottom_schools.iloc[0:5,:]

* Sort and display the five worst-performing schools

In [6]:
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [92]:
#filter by grade
grade_9 = complete_df.loc[complete_df["grade"]=="9th"]
grade_10 = complete_df.loc[complete_df["grade"]=="10th"]
grade_11 = complete_df.loc[complete_df["grade"]=="11th"]
grade_12 = complete_df.loc[complete_df["grade"]=="12th"]

#isolate scores
grade_9_math = grade_9[["school_name","math_score"]]
grade_10_math = grade_10[["school_name","math_score"]]
grade_11_math = grade_11[["school_name","math_score"]]
grade_12_math = grade_12[["school_name","math_score"]]

#group each series average by school
school_average_grade_9_math = grade_9_math.groupby(["school_name"]).mean()
school_average_grade_10_math = grade_10_math.groupby(["school_name"]).mean()
school_average_grade_11_math = grade_11_math.groupby(["school_name"]).mean()
school_average_grade_12_math = grade_12_math.groupby(["school_name"]).mean()

#rename columns
school_average_grade_9_math = school_average_grade_9_math.rename(columns={"math_score": "9th"})
school_average_grade_10_math = school_average_grade_10_math.rename(columns={"math_score": "10th"})
school_average_grade_11_math = school_average_grade_11_math.rename(columns={"math_score": "11th"})
school_average_grade_12_math = school_average_grade_12_math.rename(columns={"math_score": "12th"})

#merge dfs
combined_grade_math_average = pd.merge(school_average_grade_9_math, school_average_grade_10_math, how="outer", on="school_name")
combined_grade_math_average = pd.merge(combined_grade_math_average, school_average_grade_11_math, how="outer", on="school_name")
combined_grade_math_average = pd.merge(combined_grade_math_average, school_average_grade_12_math, how="outer", on="school_name")

#remove index label
combined_grade_math_average = combined_grade_math_average.rename_axis(None)
#format
combined_grade_math_average = round(combined_grade_math_average, 2)
combined_grade_math_average

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [93]:
#filter by grade
grade_9 = complete_df.loc[complete_df["grade"]=="9th"]
grade_10 = complete_df.loc[complete_df["grade"]=="10th"]
grade_11 = complete_df.loc[complete_df["grade"]=="11th"]
grade_12 = complete_df.loc[complete_df["grade"]=="12th"]

#isolate scores
grade_9_read = grade_9[["school_name","reading_score"]]
grade_10_read = grade_10[["school_name","reading_score"]]
grade_11_read = grade_11[["school_name","reading_score"]]
grade_12_read = grade_12[["school_name","reading_score"]]

#group each series average by school
school_average_grade_9_read = grade_9_read.groupby(["school_name"]).mean()
school_average_grade_10_read = grade_10_read.groupby(["school_name"]).mean()
school_average_grade_11_read = grade_11_read.groupby(["school_name"]).mean()
school_average_grade_12_read = grade_12_read.groupby(["school_name"]).mean()

#rename columns
school_average_grade_9_read = school_average_grade_9_read.rename(columns={"reading_score": "9th"})
school_average_grade_10_read = school_average_grade_10_read.rename(columns={"reading_score": "10th"})
school_average_grade_11_read = school_average_grade_11_read.rename(columns={"reading_score": "11th"})
school_average_grade_12_read = school_average_grade_12_read.rename(columns={"reading_score": "12th"})

#merge dfs
combined_grade_read_average = pd.merge(school_average_grade_9_read, school_average_grade_10_read, how="outer", on="school_name")
combined_grade_read_average = pd.merge(combined_grade_read_average, school_average_grade_11_read, how="outer", on="school_name")
combined_grade_read_average = pd.merge(combined_grade_read_average, school_average_grade_12_read, how="outer", on="school_name")

#remove index label
combined_grade_read_average = combined_grade_read_average.rename_axis(None)
#format
combined_grade_read_average = round(combined_grade_read_average, 2)
combined_grade_read_average

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.